## Import Modules

In [1]:
# Set paths
import os
from imp import reload

# Data manipulation
import pandas as pd
import numpy as np
from scipy import stats

# Date manipulation
import datetime as dt
import calendar
calendar.setfirstweekday(calendar.SUNDAY) 

# Custom package for data preprocessing
import mytools as mt 

# Set notebook options
pd.set_option("display.max_columns", 150)
pd.set_option("display.max_rows", 10000)
pd.set_option("display.float_format", lambda x: '%.2f' % x)

# Pretty display of multiple functions in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### File Location

In [2]:
# Data paths
import filepaths

## Load Data

In [3]:
df_master = pd.read_csv(filepaths.master_file_data, sep=',')

df_transactions = pd.read_csv(filepaths.derived_transactions_data_v1, sep=',')

In [4]:
df_transactions.head(2)

,trans_id,customer_id,trans_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type,prod_cat,prod_subcat,counter,product,returned_item,assessment_date,duration,trans_date.day,trans_date.month_num,trans_date.year,trans_date.year_month,trans_date.hour,trans_date.weekday,trans_date.week_of_year,trans_date.month,trans_date.weekday_num,trans_date.week_of_month,conversion_date,conversion_date_cohort,purchase_date_cohort
0,25890929042,266783,2011-09-23,1,2,0,0,0.00,0.00,e-Shop,Footwear,Mens,0,Footwear_Mens,yes,2014-12-02,394.00,23,9,2011,2011_09,0,06_Friday,38,09_Sep,6,month.week4,2011-09-23,2011_09,2011_09
1,98477711300,266783,2012-10-21,4,1,3,93,29.30,308.30,TeleShop,Clothing,Mens,1,Clothing_Mens,no,2014-12-02,122.00,21,10,2012,2012_10,0,01_Sunday,42,10_Oct,1,month.week3,2011-09-23,2011_09,2012_10


In [5]:
# df_transactions = df_transactions[~((df_transactions['returned_item'] == 'yes') & (df_transactions['drop_record'] == 'yes'))]

mt.check_unique_no(df_transactions, ['customer_id'])
df_transactions.shape
df_transactions.head(2)

Data has 5506 unique customer_id


(20876, 30)

,trans_id,customer_id,trans_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type,prod_cat,prod_subcat,counter,product,returned_item,assessment_date,duration,trans_date.day,trans_date.month_num,trans_date.year,trans_date.year_month,trans_date.hour,trans_date.weekday,trans_date.week_of_year,trans_date.month,trans_date.weekday_num,trans_date.week_of_month,conversion_date,conversion_date_cohort,purchase_date_cohort
0,25890929042,266783,2011-09-23,1,2,0,0,0.00,0.00,e-Shop,Footwear,Mens,0,Footwear_Mens,yes,2014-12-02,394.00,23,9,2011,2011_09,0,06_Friday,38,09_Sep,6,month.week4,2011-09-23,2011_09,2011_09
1,98477711300,266783,2012-10-21,4,1,3,93,29.30,308.30,TeleShop,Clothing,Mens,1,Clothing_Mens,no,2014-12-02,122.00,21,10,2012,2012_10,0,01_Sunday,42,10_Oct,1,month.week3,2011-09-23,2011_09,2012_10


In [6]:
df_transactions['trans_date'] = pd.to_datetime(df_transactions['trans_date'])

df_transactions['trans_date'].sort_values().min()
df_transactions['trans_date'].sort_values().max()

Timestamp('2011-01-02 00:00:00')

Timestamp('2014-12-02 00:00:00')

### RFM Customer Segmentation

In [7]:
EXTRACTION_DATE = dt.datetime(2014,12,2)

In [8]:
def categorize_customers(rfm_score):
    x = rfm_score
    if (x == '111'):
        return 'best customers'
    elif (x == '311'):
        return 'almost lost'
    elif (x == '411'):
        return 'lost customers'
    elif (x == '444'):
        return 'lost cheap customers'
    else:
        return 'other'  

In [9]:
df_rfm= df_transactions.groupby('customer_id', as_index=False).agg({'trans_date': lambda date: (EXTRACTION_DATE - date.max()).days,
                                    'trans_id': lambda num: len(num),
                                    'total_amt': lambda price: price.sum()})

In [10]:
df_rfm.rename(columns={'trans_date':'recency', 'trans_id':'frequency', 'total_amt':'monetary'}, inplace=True)

In [11]:
df_rfm['r_quartile'] = pd.qcut(df_rfm['recency'], 4, ['1','2','3','4'])
df_rfm['f_quartile'] = pd.qcut(df_rfm['frequency'], 4, ['4','3','2','1'])
df_rfm['m_quartile'] = pd.qcut(df_rfm['monetary'], 4, ['4','3','2','1'])

In [12]:
df_rfm['RFM_Score'] = df_rfm.r_quartile.astype(str)+ df_rfm.f_quartile.astype(str) + df_rfm.m_quartile.astype(str)

mt.check_unique_no(df_rfm, ['customer_id'])
df_rfm.shape

Data has 5506 unique customer_id


(5506, 8)

In [13]:
mt.check_unique_no(df_master, ['customer_id'])
df_master.shape

Data has 5506 unique customer_id


(5506, 110)

In [14]:
df_master = pd.merge(df_master, df_rfm, on='customer_id', how='left')

In [15]:
df_master['rfm_customer_segment'] = df_master['RFM_Score'].apply(categorize_customers)    

In [16]:
df_master['rfm_customer_segment'] = np.where(((df_master['rfm_customer_segment'] == 'other') & 
                                                (df_master['m_quartile'] == '1')), 'big spender',
                                           np.where(((df_master['rfm_customer_segment'] == 'other') & 
                                                     (df_master['f_quartile'] == '1')), 'loyal customers',
                                                   df_master['rfm_customer_segment']))

df_master[['customer_id', 'rfm_customer_segment']].head()

,customer_id,rfm_customer_segment
0,274213,other
1,268801,other
2,267634,best customers
3,273398,big spender
4,267592,big spender


In [17]:
df_master['rfm_customer_segment'].value_counts(dropna=False)

other                   3364
big spender              900
lost cheap customers     492
best customers           305
loyal customers          273
almost lost              131
lost customers            41
Name: rfm_customer_segment, dtype: int64

In [18]:
df_master.head(2)

,customer_id,dob,gender,city,state_code,state,region,division,customer_conversion,conversion_date,last_purchase_date,total_unique_purchase_days,total_unique_transactions,total_items_purchased,total_tax_paid,total_amt_paid,total_unique_trans_not_reversed,min_duration_btwn_purchase_days,max_duration_btwn_purchase_days,avg_duration_btwn_purchase_days,time_since_last_purchase.days,median_duration_btwn_purchase_days,assessment_date,account_age.days,account_age.years,account_age.years.group,count.Flagship store,count.MBR,count.TeleShop,count.e-Shop,sum.Flagship store,sum.MBR,sum.TeleShop,sum.e-Shop,Flagship_store_spend.prop,MBR_spend.prop,TeleShop_spend.prop,e-Shop.prop,count.Bags_Mens,count.Bags_Women,count.Books_Academic,count.Books_Children,count.Books_Comics,count.Books_DIY,count.Books_Fiction,count.Books_Non-Fiction,count.Clothing_Kids,count.Clothing_Mens,count.Clothing_Women,count.Electronics_Audio and video,count.Electronics_Cameras,count.Electronics_Computers,count.Electronics_Mobiles,count.Electronics_Personal Appliances,count.Footwear_Kids,count.Footwear_Mens,count.Footwear_Women,count.Home and kitchen_Bath,count.Home and kitchen_Furnishing,count.Home and kitchen_Kitchen,count.Home and kitchen_Tools,sum.Bags_Mens,sum.Bags_Women,sum.Books_Academic,sum.Books_Children,sum.Books_Comics,sum.Books_DIY,sum.Books_Fiction,sum.Books_Non-Fiction,sum.Clothing_Kids,sum.Clothing_Mens,sum.Clothing_Women,sum.Electronics_Audio and video,sum.Electronics_Cameras,sum.Electronics_Computers,sum.Electronics_Mobiles,sum.Electronics_Personal Appliances,sum.Footwear_Kids,sum.Footwear_Mens,sum.Footwear_Women,sum.Home and kitchen_Bath,sum.Home and kitchen_Furnishing,sum.Home and kitchen_Kitchen,sum.Home and kitchen_Tools,conversion_date.year,conversion_date.month_num,conversion_date.month,conversion_date.year_month,biological_age.actual,biological_age,biological_age.group,repeat_purchaser,returned_item_before,7_day_amt.avg,14_day_amt.avg,21_day_amt.avg,30_day_amt.avg,60_day_amt.avg,90_day_amt.avg,7_day_num.avg,14_day_num.avg,21_day_num.avg,30_day_num.avg,60_day_num.avg,90_day_num.avg,number_of_unique_times_purchases_returned,30_day_amt.avg.cumsum,cum_count,30_day_amt.avg.cumsum_pct,cum_count_pct,recency,frequency,monetary,r_quartile,f_quartile,m_quartile,RFM_Score,rfm_customer_segment
0,274213,1986-03-27,M,Houston,TX,Texas,South,West South Central,converted,2014-12-01,2014-12-01,1.00,1.00,2.00,279.72,2943.72,1.00,1.00,1.00,1.00,1.00,1.00,2014-12-02,1.00,0.00,00_01,0.00,1.00,0.00,0.00,0.00,2943.72,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2943.72,0.00,2014.00,12.00,12.0_Dec,2014.0_12.0,28.69,29.00,25_30,no,no,20606.04,41212.08,61818.12,88311.60,176623.20,264934.80,14.00,28.00,42.00,60.00,120.00,180.00,0.00,88311.60,1,0.06,0.00,1,1,2943.72,1,4,4,144,other
1,268801,1977-04-01,M,Philadelphia,PA,Pennsylvania,Northeast,Middle Atlantic,converted,2014-10-02,2014-10-02,1.00,1.00,4.00,359.94,3787.94,1.00,61.00,61.00,61.00,61.00,61.00,2014-12-02,61.00,0.17,00_01,0.00,0.00,0.00,1.00,0.00,0.00,0.00,3787.94,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3787.94,0.00,2014.00,10.00,10.0_Oct,2014.0_10.0,37.67,38.00,35_40,no,no,434.68,869.36,1304.04,1862.92,3725.84,5588.76,0.46,0.92,1.38,1.97,3.93,5.90,0.00,90174.52,2,0.07,0.00,61,1,3787.94,1,4,4,144,other


In [19]:
pd.pivot_table(df_master, index=['rfm_customer_segment'], columns=['total_unique_purchase_days'], values=['avg_duration_btwn_purchase_days'], aggfunc=['median'], margins=True, fill_value=0)

median                \
                           avg_duration_btwn_purchase_days                 
total_unique_purchase_days                             1.0    2.0    3.0   
rfm_customer_segment                                                       
almost lost                                           0.00   0.00   0.00   
best customers                                        0.00   0.00   0.00   
big spender                                           0.00 623.00 385.33   
lost cheap customers                               1045.00 607.50   0.00   
lost customers                                        0.00   0.00   0.00   
loyal customers                                       0.00   0.00   0.00   
other                                               547.50 516.50 391.67   
All                                                 853.00 548.00 390.17   

                                                                             \
                                                                              
total_unique_purchase_days    4.0    5.0    6.0    7.0    8.0    9.0   10.0   
rfm_customer_segment                                                          
almost lost                  0.00   0.00 213.50 186.14 160.31 146.17 128.45   
best customers               0.00 179.92 209.17 183.43 167.75 151.00 139.55   
big spender                300.00 254.00 213.25 186.29 166.50 150.11 135.70   
lost cheap customers         0.00   0.00   0.00   0.00   0.00   0.00   0.00   
lost customers               0.00   0.00 227.67 191.00 173.50 143.06   0.00   
loyal customers              0.00 211.00 214.25 185.29 165.62 158.22 140.10   
other                      305.88 254.60   0.00   0.00   0.00   0.00   0.00   
All                        305.38 254.40 213.50 185.79 166.50 150.67 137.20   

                                          
                                          
total_unique_purchase_days   11.0    All  
rfm_customer_segment                      
almost lost                  0.00 193.33  
best customers             124.45 183.43  
big spender                128.55 256.60  
lost cheap customers         0.00 782.00  
lost customers               0.00 220.33  
loyal customers              0.00 199.00  
other                        0.00 344.71  
All                        125.73 314.75

In [20]:
pd.pivot_table(df_master, index=['rfm_customer_segment'], columns=['total_unique_purchase_days'], values=['min_duration_btwn_purchase_days'], aggfunc=['median'], margins=True, fill_value=0)

median                    \
                           min_duration_btwn_purchase_days                     
total_unique_purchase_days                             1.0    2.0  3.0   4.0   
rfm_customer_segment                                                           
almost lost                                           0.00   0.00    0  0.00   
best customers                                        0.00   0.00    0  0.00   
big spender                                           0.00 356.00   91 56.50   
lost cheap customers                               1045.00 209.00    0  0.00   
lost customers                                        0.00   0.00    0  0.00   
loyal customers                                       0.00   0.00    0  0.00   
other                                               540.50 344.00  112 61.00   
All                                                 853.00 319.50  111 61.00   

                                                                        
                                                                        
total_unique_purchase_days 5.0 6.0   7.0   8.0   9.0  10.0 11.0    All  
rfm_customer_segment                                                    
almost lost                  0  23 22.00 12.50  5.00  3.50    0  21.00  
best customers               0  25 22.00 17.00  9.50  8.50    7  20.00  
big spender                 37  35 24.00 17.00 12.50 16.00    6  39.00  
lost cheap customers         0   0  0.00  0.00  0.00  0.00    0 782.00  
lost customers               0  19  8.50 13.00 13.50  0.00    0  17.00  
loyal customers              0  24 22.00 16.00 10.00 21.00    0  23.00  
other                       36   0  0.00  0.00  0.00  0.00    0 106.00  
All                         36  25 22.00 16.50  9.00 11.50    7  74.00

In [21]:
pd.pivot_table(df_master, index=['rfm_customer_segment'], columns=['total_unique_purchase_days'], values=['max_duration_btwn_purchase_days'], aggfunc=['median'], margins=True, fill_value=0)

median                   \
                           max_duration_btwn_purchase_days                    
total_unique_purchase_days                             1.0    2.0  3.0  4.0   
rfm_customer_segment                                                          
almost lost                                              0   0.00    0    0   
best customers                                           0   0.00    0    0   
big spender                                              0 773.50  617  559   
lost cheap customers                                  1045 905.00    0    0   
lost customers                                           0   0.00    0    0   
loyal customers                                          0   0.00    0    0   
other                                                  553 648.00  634  573   
All                                                    855 713.50  634  571   

                                                                           \
                                                                            
total_unique_purchase_days    5.0    6.0    7.0    8.0    9.0   10.0 11.0   
rfm_customer_segment                                                        
almost lost                  0.00 571.00 559.00 540.50 570.00 509.50    0   
best customers             309.00 420.00 365.00 356.00 371.50 359.50  352   
big spender                514.00 442.50 429.00 422.00 400.50 412.00  387   
lost cheap customers         0.00   0.00   0.00   0.00   0.00   0.00    0   
lost customers               0.00 781.00 749.50 715.00 699.50   0.00    0   
loyal customers            601.00 488.00 441.00 434.00 427.00 359.00    0   
other                      515.00   0.00   0.00   0.00   0.00   0.00    0   
All                        514.50 482.00 433.50 427.00 401.00 371.50  369   

                                   
                                   
total_unique_purchase_days    All  
rfm_customer_segment               
almost lost                559.00  
best customers             394.00  
big spender                500.50  
lost cheap customers       976.00  
lost customers             748.00  
loyal customers            470.00  
other                      597.50  
All                        582.00

In [22]:
mt.check_unique_no(df_master, ['customer_id'])
df_master.shape

mt.check_unique_no(df_transactions, ['customer_id'])
df_transactions.shape

Data has 5506 unique customer_id


(5506, 118)

Data has 5506 unique customer_id


(20876, 30)

## Export Data

In [23]:
df_master.to_csv(filepaths.master_file_data, index=False)
df_transactions.to_csv(filepaths.derived_transactions_data_v1, index=False)